# SmartLab project 4
### transfer learning

I couldn't find any open image downloader script, all of them had outdated image links. Therefore I downloaded pictures directly from google with [this script](https://github.com/hardikvasa/google-images-download) and made the quality check myself

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten

import numpy as np
import os
import matplotlib.pyplot as plt

In [13]:
img_height= 224
img_width= 224

In [14]:
base_dir = 'hw4'
train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')
test_dir = os.path.join( base_dir, 'test')

In [15]:
base_model = VGG19( weights = 'imagenet', include_top = False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense( 1024, activation = 'relu')(x)
predictions = Dense( 3, activation = 'softmax')(x)

model = Model( inputs = base_model.input, outputs = predictions)

In [16]:
for layer in base_model.layers:
    layer.trainable = False

model.compile( optimizer = 'adam', metrics = [ 'accuracy'], loss = 'categorical_crossentropy')

Due to the original VGG19 model my only image preprocess method is featurewise centering

In [17]:
image_datagen = ImageDataGenerator( featurewise_center = True)

train_generator = image_datagen.flow_from_directory( train_dir, target_size = ( img_height, img_width), batch_size=20)
validation_generator = image_datagen.flow_from_directory( validation_dir, target_size = ( img_height, img_width), batch_size=20)
test_generator = image_datagen.flow_from_directory( validation_dir, target_size = ( img_height, img_width), batch_size=20)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [18]:
model.fit_generator( train_generator, validation_data = validation_generator, epochs=3)

C:\Python\Python37\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
Epoch 1/3
60/60 [==============================] - 851s 14s/step - loss: 0.7938 - acc: 0.9100 - val_loss: 0.2167 - val_acc: 0.9633
Epoch 2/3
60/60 [==============================] - 1036s 17s/step - loss: 0.1367 - acc: 0.9800 - val_loss: 0.4017 - val_acc: 0.9400
Epoch 3/3
60/60 [==============================] - 1039s 17s/step - loss: 0.1094 - acc: 0.9775 - val_loss: 0.0997 - val_acc: 0.9733


In [19]:
print( "A VGG19 konvolúciós rétegei:")
for i, layer in enumerate( base_model.layers):
    print( i, layer.name)

A VGG19 konvolúciós rétegei:
0 input_5
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_conv4
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_conv4
16 block4_pool
17 block5_conv1
18 block5_conv2
19 block5_conv3
20 block5_conv4
21 block5_pool


In [20]:
for layer in model.layers[ :12]:
       layer.trainable = False
for layer in model.layers[ 12:]:
       layer.trainable = True

In [21]:
model.compile( optimizer = 'adam', metrics=[ 'accuracy'], loss = 'categorical_crossentropy')

My computer isn't strong enough to run the training (freeze every time) but it works

In [22]:
model.fit_generator( train_generator, validation_data = validation_generator, validation_steps=10, epochs=3)

Epoch 1/3
14/15 [===========================>..] - ETA: 14s - loss: 10.8221 - acc: 0.3286